In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.dates as mdates # Less verbose dates

# Let's keep the notebook user-friendly
from ipywidgets import interact, widgets, VBox, HBox 
from IPython.display import display, clear_output

In [ ]:
# Use forward slashes or relative path

df = pd.read_csv("../output/market_data_with_names_merged.csv", parse_dates=["date"])

df['date'] = pd.to_datetime(df['date'], errors='coerce')




In [ ]:
def plot_item_interactive(search_term, price_type="average"):
    clear_output(wait=True)
    # Match items containing the search term (case-insensitive)
    matched_items = df[df["type_name"].str.contains(search_term, case=False, na=False)]["type_name"].unique()
    
    if len(matched_items) == 0:
        print(f"No match found for '{search_term}'")
        return

    # If multiple, show dropdown for selection
    if len(matched_items) > 1:
        print(f"Multiple matches found for '{search_term}':")
        dropdown = widgets.Dropdown(
            options=matched_items,
            description='Select:',
            disabled=False,
        )
        display(dropdown)
        def dropdown_plot(selected_item):
            _plot_item(selected_item, price_type)
        interact(dropdown_plot, selected_item=dropdown)
    else:
        _plot_item(matched_items[0], price_type)

def _plot_item(item, price_type):
    item_df = df[df["type_name"] == item].sort_values("date")
    plt.figure(figsize=(12, 4))
    ax = plt.gca()
    item_df.groupby("date")[price_type].mean().plot()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.title(f"{item} - {price_type.title()} Price Over Time")
    plt.xlabel("Date")
    plt.ylabel(f"{price_type.title()} Price")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    print(f"Showing: {item}")


In [ ]:
search_box = widgets.Text(
    value='hydrogen',
    placeholder='Type part of an item name...',
    description='Search:',
    disabled=False
)

price_type_dropdown = widgets.Dropdown(
    options=[
        ('Average Price', 'average'),
        ('Highest Price', 'highest'),
        ('Lowest Price', 'lowest'),
        ('Volume', 'volume'),
        ('Order Count', 'order_count')
    ],
    value='average',
    description='Show:',
    disabled=False,
)


ui = VBox([search_box, price_type_dropdown])

def update_plot(search_term, price_type):
    plot_item_interactive(search_term, price_type)

out = widgets.interactive_output(update_plot, {'search_term': search_box, 'price_type': price_type_dropdown})
display(ui, out)


In [ ]:
print(df.columns)